In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-17 23:05:50.910057: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 23:05:51.046424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 23:05:51.046531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 23:05:51.049126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alre

In [2]:
# Load data from a single file
def load_data(data_path):
    df = pd.read_csv(data_path)
    data = df['filename'].tolist()
    labels = df['label'].tolist()
    return data, labels

In [3]:
# Preprocess audio data
def preprocess_audio(file_path, target_duration=10):
    audio, _ = librosa.load(file_path, sr=None)

    target_length = int(target_duration * _)
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    else:
        audio = audio[:target_length]

    mfccs = librosa.feature.mfcc(y=audio, sr=_, n_mfcc=13)
    return mfccs

In [4]:
def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
def train_model(X_train, y_train, X_val, y_val):
    model = build_model(input_shape=X_train.shape[1:])
    optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Data Augmentation
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
    datagen.fit(X_train)

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Model Training
    model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=20, callbacks=[early_stopping], verbose=1)

    # Save the trained model to a file
    model.save('temp_hindi_hate_speech_model.h5')

    return model

In [11]:
# Main function
def main():

    train_data_path = 'Hindi_train.csv'
    test_data_path = 'Hindi_test.csv'

    # Load training data
    data_train, labels_train = load_data(train_data_path)

    # Load testing data
    data_test, labels_test = load_data(test_data_path)

    # Convert string labels to integers
    y_train = np.array([1 if label == 'Yes' else 0 for label in labels_train])
    y_test = np.array([1 if label == 'Yes' else 0 for label in labels_test])

    # Combine datasets for k-fold cross-validation
    data_combined = data_train + data_test
    labels_combined = y_train.tolist() + y_test.tolist()

    # Convert to numpy arrays
    X_combined = np.array([preprocess_audio(os.path.join('./audios/', filename)) for filename in tqdm(data_combined, desc="Processing Audio")])
    y_combined = np.array(labels_combined)

    # Initialize cross-validator
    n_splits = 5  # You can adjust the number of folds
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold = 1
    accuracies = []
    all_true_labels = []
    all_pred_labels = []

    for train_index, val_index in skf.split(X_combined, y_combined):
        X_train, X_val = X_combined[train_index], X_combined[val_index]
        y_train, y_val = y_combined[train_index], y_combined[val_index]

        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Train the model
        model = train_model(X_train, y_train, X_val, y_val)

        # Predict on the validation set
        y_pred_probs = model.predict(X_val)
        y_pred = (y_pred_probs > 0.5).astype(int)

        # Evaluate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Fold {fold} - Validation Accuracy: {accuracy * 100:.2f}%")

        # Collect true and predicted labels for confusion matrix
        all_true_labels.extend(y_val)
        all_pred_labels.extend(y_pred.flatten())

        accuracies.append(accuracy)
        fold += 1

    # Save the final trained model with date and time in the name, in 'final_model' directory
    current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_model_filename = f"final_model_{current_datetime}.h5"
    
    model.save(final_model_filename)
    print(f"\nFinal trained model saved as {final_model_filename}")

    # Calculate and print the overall accuracy
    overall_accuracy = np.mean(accuracies)
    print(f"\nOverall Cross-Validation Accuracy: {overall_accuracy * 100:.2f}%")

    # Calculate and print the confusion matrix
    cm = confusion_matrix(all_true_labels, all_pred_labels)
    print("\nConfusion Matrix:")
    print(cm)

    print("Cross-validation completed.")

In [12]:
if __name__ == "__main__":
    main()

Processing Audio:   0%|          | 2/1191 [00:00<01:35, 12.39it/s]

Processing Audio: 100%|██████████| 1191/1191 [01:37<00:00, 12.15it/s]


Epoch 1/20
30/30 [==============================] - 8s 173ms/step - loss: 1.4941 - accuracy: 0.6092 - val_loss: 2.1154 - val_accuracy: 0.7406
Epoch 2/20
30/30 [==============================] - 5s 157ms/step - loss: 1.3601 - accuracy: 0.6124 - val_loss: 2.2309 - val_accuracy: 0.7238
Epoch 3/20
30/30 [==============================] - 4s 147ms/step - loss: 1.4169 - accuracy: 0.6218 - val_loss: 1.1897 - val_accuracy: 0.7406
Epoch 4/20
30/30 [==============================] - 5s 161ms/step - loss: 1.4563 - accuracy: 0.6008 - val_loss: 2.5033 - val_accuracy: 0.6695
Epoch 5/20
30/30 [==============================] - 5s 170ms/step - loss: 1.0770 - accuracy: 0.6197 - val_loss: 1.4715 - val_accuracy: 0.6485
Epoch 6/20
30/30 [==============================] - 5s 153ms/step - loss: 1.0501 - accuracy: 0.6208 - val_loss: 1.1910 - val_accuracy: 0.7573


/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 44ms/step
Fold 1 - Validation Accuracy: 74.06%
Epoch 1/20
30/30 [==============================] - 9s 180ms/step - loss: 1.4778 - accuracy: 0.6222 - val_loss: 4.0180 - val_accuracy: 0.6597
Epoch 2/20
30/30 [==============================] - 5s 156ms/step - loss: 1.5285 - accuracy: 0.6159 - val_loss: 2.5373 - val_accuracy: 0.6345
Epoch 3/20
30/30 [==============================] - 4s 144ms/step - loss: 1.4136 - accuracy: 0.6107 - val_loss: 1.9338 - val_accuracy: 0.7101
Epoch 4/20
30/30 [==============================] - 5s 165ms/step - loss: 1.6123 - accuracy: 0.5834 - val_loss: 3.7097 - val_accuracy: 0.6681
Epoch 5/20
30/30 [==============================] - 4s 142ms/step - loss: 1.3711 - accuracy: 0.6086 - val_loss: 1.0100 - val_accuracy: 0.7017
Epoch 6/20
30/30 [==============================] - 4s 135ms/step - loss: 1.1042 - accuracy: 0.6285 - val_loss: 1.1652 - val_accuracy: 0.7227
Epoch 7/20
30/30 [==============================] - 4s 143m

...

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 42ms/step
Fold 4 - Validation Accuracy: 69.33%
Epoch 1/20
30/30 [==============================] - 8s 169ms/step - loss: 1.2930 - accuracy: 0.5897 - val_loss: 3.0990 - val_accuracy: 0.6597
Epoch 2/20
30/30 [==============================] - 5s 157ms/step - loss: 1.4691 - accuracy: 0.6149 - val_loss: 1.2732 - val_accuracy: 0.6933
Epoch 3/20
30/30 [==============================] - 5s 157ms/step - loss: 1.4269 - accuracy: 0.6170 - val_loss: 3.4568 - val_accuracy: 0.5966
Epoch 4/20
30/30 [==============================] - 5s 165ms/step - loss: 1.3724 - accuracy: 0.5929 - val_loss: 2.8228 - val_accuracy: 0.6134
Epoch 5/20
30/30 [==============================] - 5s 151ms/step - loss: 1.2467 - accuracy: 0.6065 - val_loss: 1.0179 - val_accuracy: 0.7227
Epoch 6/20
30/30 [==============================] - 5s 153ms/step - loss: 1.1235 - accuracy: 0.6139 - val_loss: 1.1740 - val_accuracy: 0.6597
Epoch 7/20
30/30 [==============================] - 5s 160m

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 38ms/step
Fold 5 - Validation Accuracy: 72.27%


AttributeError: 'str' object has no attribute 'keras'